In [1]:
import pandas as pd 
import numpy as np

ЗАДАНИЕ 1. БАЗОВОЕ ИЗУЧЕНИЕ 

Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения 
Провести расчет базовых метрик для них, кратко описать результаты.

Изучив данный датасет, можно сделать предварительнный вывод (пока без расчёта базовых статистик) о том, что большинство 
лошадей взрослые особи. В столбце Age помимо 1 также фигурирует цифра 9, но нет 2 (жеребёнок). Исходя из этого можно предположить, что это ошибка ввода данных и скорее всего имелось ввиду, что это молодые особи (< 6 лет). Далее заменим цифру 9 на 2. У большинства лошадей температура и пульс выше нормальных значений, из чего следует, что большинство особей больны. Температура ниже нормального значения говорит о том, что лошадь пребывает в состоянии late shock, а значит тоже больна 
Ниже будут указаны остальные характеристики 

In [2]:
df = pd.read_csv('horse_data.csv', na_values = '?')
df.iloc[:, [0, 1, 3, 4, 5, 6, 10, 22]] # выводим нужные столбцы по индексу, далее переименовываем
df1 = df.rename(columns={'2': 'surgery', '1': 'age', '38.50': 'rectal temperature','66': 'pulse', '28': 'respiratory rate','3': 'temperature of extremities', '5': 'pain', '2.2': 'outcome'})
poor_horses = df1[['surgery', 'age', 'rectal temperature', 'pulse', 'respiratory rate','temperature of extremities', 'pain', 'outcome']]
poor_horses.head(10)


,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
6,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
7,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
8,2.0,9,38.3,90.0,NaN,1.0,5.0,1.0
9,1.0,1,38.1,66.0,12.0,3.0,3.0,1.0


ЗАДАНИЕ 2. ПЕРВИЧНОЕ ИЗУЧЕНИЕ ДАННЫХ 

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

Среднее значение ректальной температуры - 38.17, что превышает нормальное значение 37.8 ; так же видим, что среднее значение пульса почти в два раза превышает допустимое нормальное значение. Значительно превышен показатель частоты дыхания - 30.43. 
Повышенные средние значения говорят нам о том, что большинство лошадей были больны и прооперированы. 

In [15]:
poor_horses.describe()

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,298.000000,299.00000,239.000000,275.000000,241.000000,243.000000,244.000000,298.000000
mean,1.395973,1.64214,38.166527,71.934545,30.427386,2.345679,2.942623,1.550336
std,0.489881,2.17730,0.733508,28.680522,17.678256,1.046369,1.303993,0.737967
min,1.000000,1.00000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.00000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.00000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.00000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.00000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


Трактовка модальных значений:

Модальные значения категориальных показателей говорят нам о том, что большинство лошадей перенесли хирургическое вмешательство, после чего животное не умирало, а продолжало жить. Модальный показатель pain говорит нам о том, что большинство лошадей испытавали не тяжелую (severe), а лёгкую прерывистую боль


In [66]:
print('Значения моды: \n surgery: ', poor_horses['surgery'].round().mode()[0],'\n outcome:', 
poor_horses['outcome'].round().mode()[0],'\n age:', poor_horses['age'].round().mode()[0],'\n pain:', poor_horses['pain'].round().mode()[0],'\n temperature of extremities:', poor_horses['temperature of extremities'].round().mode()[0])

Значения моды: 
 surgery:  1.0 
 outcome: 1.0 
 age: 1 
 pain: 3.0 
 temperature of extremities: 3.0


2.1 ВЫБРОСЫ 

Найти выбросы, выдвинуть предположения откуда они появились и принять решение, что с ними делать 
(будем искать через межквартильный размах)

2.1.1 AGE

Отсортировав наш датасет по столбцу age мы видим, что у нас представлены значения только 1 и 9. Теперь мы можем подтвердить нашу гипотезу о том, что цифрой 9 отмечены те самые молодые особи, возрастом < 6 лет. Ниже заменим значение 9 на 2

In [22]:
sorted_df = poor_horses.sort_values(by='age', ascending=False)
sorted_df.head(10)

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
190,2.0,9,NaN,100.0,44.0,2.0,4.0,1.0
251,1.0,9,38.8,124.0,36.0,3.0,2.0,2.0
274,1.0,9,38.8,150.0,50.0,1.0,5.0,2.0
108,1.0,9,38.4,84.0,40.0,3.0,3.0,2.0
134,1.0,9,38.1,136.0,48.0,3.0,5.0,2.0
102,1.0,9,38.0,140.0,68.0,1.0,3.0,1.0
38,1.0,9,39.2,146.0,96.0,NaN,NaN,2.0
22,1.0,9,38.3,130.0,60.0,NaN,2.0,1.0
211,1.0,9,38.9,120.0,30.0,1.0,3.0,1.0
73,1.0,9,NaN,NaN,NaN,NaN,NaN,2.0


In [17]:
q1 = poor_horses['age'].quantile(0.25)
q3 = poor_horses['age'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['age'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
print('Кол-во выбросов: ', len(remove_outliers['age']))


Кол-во выбросов:  24


In [28]:
poor_horses.loc[poor_horses['age'] == 9, 'age'] = 2
poor_horses.head(10)

C:\Users\romab\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
5,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
6,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
7,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
8,2.0,2,38.3,90.0,NaN,1.0,5.0,1.0
9,1.0,1,38.1,66.0,12.0,3.0,3.0,1.0


2.1.2 SURGERY

In [30]:
q1 = poor_horses['surgery'].quantile(0.25)
q3 = poor_horses['surgery'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['surgery'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
print('Кол-во выбросов: ', len(remove_outliers['surgery']))
remove_outliers['surgery']

Кол-во выбросов:  1


131   NaN
Name: surgery, dtype: float64

2.1.3 OUTCOME

In [33]:
q1 = poor_horses['outcome'].quantile(0.25)
q3 = poor_horses['outcome'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['outcome'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
print('Кол-во выбросов: ', len(remove_outliers['outcome']))
remove_outliers['outcome']

Кол-во выбросов:  1


131   NaN
Name: outcome, dtype: float64

2.1.4 RECTAL TEMPERATURE (ректальная температура)

Выбросы с данными значенями вполне допустимы для больной лошади. Оставляем их 

In [35]:
q1 = poor_horses['rectal temperature'].quantile(0.25)
q3 = poor_horses['rectal temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['rectal temperature'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
remove_outliers['rectal temperature']

4       NaN
6       NaN
7       NaN
15      NaN
19     39.9
       ... 
273     NaN
280    40.0
292     NaN
294     NaN
297    36.5
Name: rectal temperature, Length: 61, dtype: float64

2.1.5 PULSE

Здесь мы наблюдаем, что высокие показатели пульса (они же выбросы) характерны для больных жеребят. Поэтому не считаем это ошибкой, оставляем как есть 

In [39]:
q1 = poor_horses['pulse'].quantile(0.25)
q3 = poor_horses['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['pulse'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
remove_outliers[['age','pulse']]

,age,pulse
2,2,164.0
4,1,NaN
40,2,150.0
51,1,NaN
54,2,160.0
57,1,NaN
73,2,NaN
92,1,NaN
114,1,NaN
116,1,NaN


2.1.6 RESPIRATORY RATE/ ЧАСТОТА ДЫХАНИЯ

Вполне допустимые (высокие) показатели, считаю нужным оставить 

In [42]:
q1 = poor_horses['respiratory rate'].quantile(0.25)
q3 = poor_horses['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
remove_outliers[['age','respiratory rate']]

,age,respiratory rate
2,2,84.0
4,1,NaN
6,1,NaN
8,2,NaN
28,1,NaN
...,...,...
270,1,NaN
273,1,NaN
280,1,NaN
283,1,NaN


2.1.7 TEMPERATURE OF EXTREMITIES / ТЕМПЕРАТУРА В КОНЕЧНОСТЯХ

выбросы - все значения NaN

In [45]:
q1 = poor_horses['temperature of extremities'].quantile(0.25)
q3 = poor_horses['temperature of extremities'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['temperature of extremities'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
print('Кол-во выбросов: ', len(remove_outliers['temperature of extremities']))
# remove_outliers['temperature of extremities']

Кол-во выбросов:  51


2.1.8 PAIN

выбросы - все значения NaN

In [48]:
q1 = poor_horses['pain'].quantile(0.25)
q3 = poor_horses['pain'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = poor_horses[poor_horses['pain'].between(lower_bound, upper_bound, inclusive=True)]
remove_outliers

remove_outliers = pd.concat([poor_horses, remove_outliers]).drop_duplicates(keep=False)
print('Кол-во выбросов: ', len(remove_outliers['pain']))
# remove_outliers['pain']

Кол-во выбросов:  46


ЗАДАНИЕ 3. РАБОТА С ПРОПУСКАМИ 

Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

3.1 Столбцы 'surgery' и 'outcome'. 
Выше мы видели, что в данных столбцах есть по 1-му выбросу в строке 132. На фоне общего количества данных приняла решение удалить этот выброс, т.к считаю, что на итоговых показателях удаление никак не скажется. 
Более того, не совсем очевидно каким значением его можно было бы заменить 

In [49]:
poor_horses_update = poor_horses.drop(poor_horses.index[132])
poor_horses_update

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


3.2. Ректальная температура. 

В зависимости от тяжести протекания болезни и от возраста животного, температура может быть как выше, так и ниже среднего значения. Считаю, что уместо заменить выбросы срредним значением 

In [53]:
poor_horses_update['rectal temperature'].fillna(poor_horses_update['rectal temperature'].median(), inplace=True)
poor_horses_update

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,38.2,NaN,NaN,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


3.3. Pulse / пульс 

У взрослых особей и жеребях показатели пульса существенно различаются. Поэтому принято решения заменить пропуски медианой, предварительно сгруппировав по столбцу age 

In [59]:
poor_horses_update['pulse'].fillna(poor_horses_update.groupby('age')['pulse'].transform('median'), inplace=True)
poor_horses_update
#print(poor_horses_update.groupby(['age', 'pulse'])['pulse'].median())

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,38.2,60.0,NaN,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


3.4. Respiratory rate / частота дыхания 

Так как нормальный показатель 8-10 встречается только лишь у двух взрослых лошадей из всего кол-ва особей, и в целом, показатель частоты дыхания существенно превышает норму как у взрослых, так и у жеребят, приняла решения заменить пропуски среднем значением mean 

In [63]:
poor_horses_update['respiratory rate'].fillna(poor_horses_update['respiratory rate'].median(), inplace=True)
poor_horses_update
#print(poor_horses_update.groupby(['age', 'respiratory rate'])['respiratory rate'].median())

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,38.2,60.0,24.5,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


3.5. Temperature of extremities / температура конечностей и Pain / боль 

Так как у двух данных показателей совпадают значения моды, приняла решения заменить пропуски в обоих столбцах модальным значением 
Выше было описано, что большинство лошадей (взр и жеребят) имели прерывистую легкую боль, что как раз-таки соотвествует значению моды - 3

In [71]:
poor_horses_update['temperature of extremities'].fillna(poor_horses_update['temperature of extremities'].mode()[0], inplace=True)
poor_horses_update['pain'].fillna(poor_horses_update['pain'].mode()[0], inplace=True)
poor_horses_update

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,3.0,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,3.0,3.0,2.0
4,2.0,1,38.2,60.0,24.5,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...
294,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
295,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
296,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
297,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


СПАСИБО БОЛЬШОЕ! КЛАССНОЕ ЗАДАНИЕ :) 